<img src= "http://www.voyagerconsulting.org/images/logo1.png">

# Voyager Consulting: Jira API

## Import Libraries

In [1]:
import requests
import json
import pandas as pd
import time
import datetime

In [2]:
conda install jira

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


## Configurations

In [3]:
from collections import Counter
from jira import JIRA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
#Will connect to a Jira Instance started from the Atlassian Plugin SDK
jira = JIRA(basic_auth=("admin", "admin")) # a username/password tuple

# Get the mutable application properties for this server (requires
# jira-system-administrators permission)
props = jira.application_properties()

#Oauth
key_cert_data = None
with open(key_cert, 'r') as key_cert_file:
    key_cert_data = key_cert_file.read()

oauth_dict = {
    'access_token': 'foo',
    'access_token_secret': 'bar',
    'consumer_key': 'jira-oauth-consumer',
    'key_cert': key_cert_data
}
auth_jira = JIRA(oauth=oauth_dict)

In [14]:
#Everything goes through the JIRA object
jira = JIRA('https://plutotv.atlassian.net')

## Issues

### Getting Issues

In [ ]:
issue = jira.issue('JRA-1330')

#Get direct fields of issues
summary = issue.fields.summary         # 'Field level security permissions'
votes = issue.fields.votes.votes       # 440 (at least)

#Get specific fields
issue = jira.issue('JRA-1330', fields='summary,comment')

### Searching Issues with JQL

In [ ]:
# Search returns first 50 results, `maxResults` must be set to exceed this
issues_in_proj = jira.search_issues('project=PROJ')
all_proj_issues_but_mine = jira.search_issues('project=PROJ and assignee != currentUser()')

# my top 5 issues due by the end of the week, ordered by priority
oh_crap = jira.search_issues('assignee = currentUser() and due < endOfWeek() order by priority desc', maxResults=5)

# Summaries of my last 3 reported issues
for issue in jira.search_issues('reporter = currentUser() order by created desc', maxResults=3):
    print('{}: {}'.format(issue.key, issue.fields.summary))

## Projects

In [ ]:
projects = jira.projects()
jra = jira.project('JRA')
print(jra.name)                 # 'JIRA'
print(jra.lead.displayName)     # 'John Doe [ACME Inc.]'

In [ ]:
##THESE ARE ALL UNDER THE JIRA OBJECT
#boards
#issue
#sprint
#sprint_info
#sprints_by_name
#statuscategories()
#user

## Something about visualizations

In [ ]:
# Read data from Jira with changelog
jira_search = jira.search_issues(jql, startAt=block_num*block_size, maxResults=block_size, 
                                 fields="issuetype, created, resolutiondate, reporter, assignee, status", 
                                 expand='changelog')

# Get information from changelog
history_assignee = []
histories = issue.raw['changelog'].get('histories', None)
if histories is not None:
    for history in histories:
        for item in history['items']:
            if item['field'] == 'assignee':
                # Get history author, previous assignee, new assignee
                history_author = history.get('author', None)
                if history_author is not None:
                    history_author = history_author['key']
                history_assignee.append([history_author, item['from'], item['to'], datetime.strptime(history['created'][:19], "%Y-%m-%dT%H:%M:%S")])
